# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

### Importando a base de dados
Importamos e mostramos a base de dados como recebemos.

In [23]:
#Passo 02: Importar a base de dados
import pandas as pd
tabela = pd.read_excel("commodities.xlsx")
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


### Pesquisando o preço atual dos produtos
Aqui pesquisamos o preço atual de todos os produtos no google chrome,<br>
usamos o selenium para fazer as pesquisas, logo após já atualizamos o<br>
preço na base de dados.

In [24]:
#webdriver
#firefox -> geckodriver
#chrome -> chromedriver
from selenium import webdriver
navegador = webdriver.Chrome()
#Utilizamos o for para repetir os mesmos passo pra todos os produtos
for linha in tabela.index:
    produto = tabela.loc[linha, "Produto"]
    #print(produto)
    #Usamos o .replace() para substituir os caracteres especiais por letras que serão aceitas nos links  
    produto = produto.replace("ó","o").replace("ã","a").replace("ç","c").replace("ú","u").replace(
        "é","e").replace("á","a")
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    navegador.get(link)
    #Aqui pegamos os preços atualizado dos produtos no mercado
    preco = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    #Formatamos os números para o padrão (apenas com o separador de ponto)
    preco = preco.replace(".","").replace(",",".")
    #Transformando os dados recebidos em float
    tabela.loc[linha, "Preço Atual"] = float(preco)
    #print(preco)
display(tabela)
#Comandos do atribuídos ao .find_element():
#.click() vai clikar
#.send_keys("texto") -> escrecer
#.get_attribute() -> pegar o valor
#Sempre colocar o xpath em aspas simples

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,NaN
1,Soja,163.59,162.64,NaN
2,Boi,282.20,276.05,NaN
3,Petróleo,424.37,393.26,NaN
4,Algodão,497.76,498.88,NaN
5,Açúcar,136.23,134.64,NaN
6,Café,1092.87,1087.87,NaN
7,Ouro,321.77,327.99,NaN
8,Trigo,1549.11,1546.08,NaN
9,Tilápia,9.05,9.05,NaN


### Preenchendo a coluna de compra
Caso o preço atual for igual ou nemor que o preço ideal devemos <br>
comprar (True) caso contrário então não devemos comprar (False).

In [25]:
# Passo 06: Descidir quais produtos comprar
#Preencher a coluna de comprar
tabela["Comprar"] = tabela["Preço Atual"] <= tabela["Preço Ideal"]
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,False
1,Soja,163.59,162.64,True
2,Boi,282.20,276.05,True
3,Petróleo,424.37,393.26,True
4,Algodão,497.76,498.88,False
5,Açúcar,136.23,134.64,True
6,Café,1092.87,1087.87,True
7,Ouro,321.77,327.99,False
8,Trigo,1549.11,1546.08,True
9,Tilápia,9.05,9.05,False


### Exportando a base de dados para Exel
Exportamos a base de dados como os dados atualizados<br>
para a pasta que estamos utilizando no momento, no caso WebScraping.

In [26]:
#Passo 07: Exportat a base para exel
tabela.to_excel("commodities_atualizado.xlsx", index=False)
navegador.quit()